In [ ]:
import os
import pandas as pd
import numpy as np
import sklearn as sk
import warnings
import itertools as IT
import glob
import datetime as dt
import time
import sys

import lightgbm as lgb
from scipy import stats

from dateutil.relativedelta import relativedelta

# from xgboost import plot_importance
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import RobustScaler


# #Common Model Algorithms
# from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
# import xgboost

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',500)
pd.set_option('max_info_columns',500)
pd.set_option('display.float_format', lambda x : '%.3f' % x )

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
#from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss
#from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold


from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

# Use GridSearchCV to find the best parameters.
from sklearn.model_selection import GridSearchCV
import xgboost
from sklearn.model_selection import cross_val_score

test= pd.read_csv("../input/fire-gimhae/PJT002_test.csv")
train = pd.read_csv("../input/fire-gimhae/PJT002_train.csv")
val = pd.read_csv("../input/fire-gimhae/PJT002_validation.csv")

print(train.shape)
print(val.shape)
print(test.shape)


train['target_value']=train.fr_yn.apply(lambda x: 1 if x=="Y" else 0)
val['target_value']=val.fr_yn.apply(lambda x: 1 if x=="Y" else 0)


train.describe()
test.describe()

def crosstab_ratio(data,x):
    if data[x].dtype == 'object' :
        print('Fire Crosstab by : x')
        print(pd.crosstab(data[x],data['fr_yn']).apply(lambda r: r/r.sum(), axis=1))
        print('-'*10, '\n')
        
def crosstab(data,x):
    if data[x].dtype == 'object' :
        print('Fire Crosstab by : x')
        print(pd.crosstab(data[x],data['fr_yn']))
        print('-'*10, '\n')
        
        



# 전처리

## 건물

object_list=[]

for col in train:
    if train[col].dtype =="object":
        object_list.append(col)
                

        
object_train=train[object_list[:]]

object_train.head(5)



object_list=[]

for col in val:
    if val[col].dtype =="object":
        object_list.append(col)
        
object_val=val[object_list[:]]

object_list=[]

for col in test:
    if test[col].dtype =="object":
        object_list.append(col)
        
object_test=test[object_list[:]]

object_test.head(5)
                

print(len(object_val.columns))
print(len(object_train.columns))
print(set(object_train)-set(object_val))
print(set(object_train)-set(object_test))

# 이거 빼고 다 일치

> ### bldng_us(건물용도)

# train엔 있고 val엔 없는 데이터 출력 
# vice versa

print(set(object_train["bldng_us"].unique())-set(object_val["bldng_us"].unique()))
print(set(object_val["bldng_us"].unique())-set(object_train["bldng_us"].unique()))

print(set(object_train["bldng_us"].unique())-set(object_test["bldng_us"].unique()))
print(set(object_test["bldng_us"].unique())-set(object_train["bldng_us"].unique()))

only_train_bldng_us=list(set(object_train["bldng_us"].unique())-set(object_test["bldng_us"].unique()))

# 결측치 탐색

print(object_train[object_train["bldng_us"].isna()].shape)

print(object_val[object_val["bldng_us"].isna()].shape)

print(object_test[object_test["bldng_us"].isna()].shape)

object_train.groupby("bldng_us").size().sort_values(ascending=False)

object_val.groupby("bldng_us").size().sort_values(ascending=False)

object_test.groupby("bldng_us").size().sort_values(ascending=False)

crosstab(train,"bldng_us")

crosstab(val,"bldng_us")


object_train.drop(object_train["bldng_us"].isin(only_train_bldng_us).index)

# drop할지, None 할지 두고 보기

object_train.drop(object_train[object_train["bldng_us"].isin(only_train_bldng_us)].index,axis=0,inplace=True)




bldng_us_dummies = pd.get_dummies(object_train["bldng_us"])
object_train = object_train.join(bldng_us_dummies)

bldng_us_dummies = pd.get_dummies(object_val["bldng_us"])
object_val = object_val.join(bldng_us_dummies)

bldng_us_dummies = pd.get_dummies(object_test["bldng_us"])
object_test = object_test.join(bldng_us_dummies)


object_train['bldng_archtctr'][object_train['bldng_archtctr'].isin(['기타콘크리트구조','통나무구조','프리케스트콘크리트구조'])]=None

low = ["목구조", "일반목구조", "벽돌구조", "블록구조", "석구조", "조적구조"]
normal = ["강파이프구조", "경량철골구조", "기타강구조", "기타구조", "기타조적구조"]
high = ["일반철골구조", "철골철근콘크리트구조", "철골콘크리트구조", "철근콘크리트구조"]


object_train['bldng_archt_danger']=object_train.bldng_archtctr.apply(lambda x: "low" if x in low else x)
object_train['bldng_archt_danger']=object_train.bldng_archt_danger.apply(lambda x: "normal" if x in normal else x)
object_train['bldng_archt_danger']=object_train.bldng_archt_danger.apply(lambda x: "high" if x in high else x)

object_val['bldng_archt_danger']=object_val.bldng_archtctr.apply(lambda x: "low" if x in low else x)
object_val['bldng_archt_danger']=object_val.bldng_archt_danger.apply(lambda x: "normal" if x in normal else x)
object_val['bldng_archt_danger']=object_val.bldng_archt_danger.apply(lambda x: "high" if x in high else x)


object_test['bldng_archt_danger']=object_test.bldng_archtctr.apply(lambda x: "low" if x in low else x)
object_test['bldng_archt_danger']=object_test.bldng_archt_danger.apply(lambda x: "normal" if x in normal else x)
object_test['bldng_archt_danger']=object_test.bldng_archt_danger.apply(lambda x: "high" if x in high else x)

bldng_archt_dummies = pd.get_dummies(object_train["bldng_archt_danger"])

object_train = object_train.join(bldng_archt_dummies)

bldng_archt_dummies = pd.get_dummies(object_val["bldng_archt_danger"])
object_val = object_val.join(bldng_archt_dummies)

bldng_archt_dummies = pd.get_dummies(object_test["bldng_archt_danger"])
object_test = object_test.join(bldng_archt_dummies)

### dt_of_athrztn 건물승인날짜

object_val["dt_of_athrztn"]=val['dt_of_athrztn']
object_test["dt_of_athrztn"]=test['dt_of_athrztn']

object_train.dt_of_athrztn=object_train.dt_of_athrztn.apply(lambda x: str(x)[0:4] if len(str(x))==10 else x)
object_train.dt_of_athrztn=object_train.dt_of_athrztn.apply(lambda x: str(x)[0:4] if len(str(x))==8 else x)
object_train.dt_of_athrztn=object_train.dt_of_athrztn.apply(lambda x: str(x)[0:4] if len(str(x))==6 else x)

object_val["dt_of_athrztn"] = object_val["dt_of_athrztn"].astype(str).str[:4]
object_test["dt_of_athrztn"]=object_test["dt_of_athrztn"].astype(str).str[:4]

object_train.dt_of_athrztn

dirty_date_idx=object_train.dt_of_athrztn[object_train.dt_of_athrztn.astype(str).apply(len)>4].index

object_train.dt_of_athrztn[dirty_date_idx[0]]="1980"
object_train.dt_of_athrztn[dirty_date_idx[1]]="1890"
object_train.dt_of_athrztn[dirty_date_idx[2]]="1890"
object_train.dt_of_athrztn[dirty_date_idx[3]]="1990"
object_train.dt_of_athrztn[dirty_date_idx[4]]="1990"
object_train.dt_of_athrztn[dirty_date_idx[5]]="1982"
object_train.dt_of_athrztn[dirty_date_idx[6]]="1978"
object_train.dt_of_athrztn[dirty_date_idx[7]]="1994"

dirty_date_idx2=object_train.dt_of_athrztn[(object_train.dt_of_athrztn.notnull())&(object_train.dt_of_athrztn.astype(str).apply(len)<4)].index

#결측치가 아닌 연도 데이터를 시각화 하는 코드

#sns.distplot(object_data.dt_of_athrztn[~(object_data.dt_of_athrztn.isna)()].astype(int))

말도안돼_연도=object_train.dt_of_athrztn[~(object_train.dt_of_athrztn.isna)()].astype(int)[object_train.dt_of_athrztn[~(object_train.dt_of_athrztn.isna)()].astype(int)>2019].index

object_train.dt_of_athrztn=object_train.dt_of_athrztn.apply(lambda x : "1997" if x=="9712" else x)

object_train.dt_of_athrztn.loc[53193]

있는_연도=object_train.dt_of_athrztn[~(object_train.dt_of_athrztn.isna())].astype(int)

not_na_idx=object_train.dt_of_athrztn[object_train.dt_of_athrztn.notnull()].index

object_train.dt_of_athrztn[not_na_idx]=object_train.dt_of_athrztn[not_na_idx].astype(int)

object_train["Bldng_Age"]=object_train.dt_of_athrztn.apply(lambda x: 2019-x)

year_median=object_train["Bldng_Age"][object_train["Bldng_Age"].notnull()].median()

object_train["Bldng_Age"].fillna(year_median,inplace=True)

object_val.loc[(object_val["dt_of_athrztn"].isna()),"dt_of_athrztn"]= object_val[~(object_val["dt_of_athrztn"].isna())]["dt_of_athrztn"].median()

object_test.loc[(object_test["dt_of_athrztn"].isna()),"dt_of_athrztn"]= object_test[~(object_test["dt_of_athrztn"].isna())]["dt_of_athrztn"].median()

object_val.dt_of_athrztn.replace("nan",object_val[~(object_val["dt_of_athrztn"].isna())]["dt_of_athrztn"].median(),inplace=True)
object_test.dt_of_athrztn.replace("nan",object_test[~(object_test["dt_of_athrztn"].isna())]["dt_of_athrztn"].median(),inplace=True)


object_val['Bldng_Age']=object_val.dt_of_athrztn.astype(int)
object_test['Bldng_Age']=object_test.dt_of_athrztn.astype(int)

object_val['Bldng_Age']=object_val['Bldng_Age'].apply(lambda x: 2019-x)
object_test['Bldng_Age']=object_test['Bldng_Age'].apply(lambda x: 2019-x)

criterion_age=20

object_train.loc[object_train["Bldng_Age"]<criterion_age,"Bldng_old_YN"] = "old"
object_train.loc[object_train["Bldng_Age"]>=criterion_age,"Bldng_old_YN"] = "new"

object_val.loc[object_val["Bldng_Age"]<criterion_age,"Bldng_old_YN"] = "old"
object_val.loc[object_val["Bldng_Age"]>=criterion_age,"Bldng_old_YN"] = "new"

object_test.loc[object_test["Bldng_Age"]<criterion_age,"Bldng_old_YN"] = "old"
object_test.loc[object_test["Bldng_Age"]>=criterion_age,"Bldng_old_YN"] = "new"

In [ ]:
# rd_sd_nm

# train엔 있고 val엔 없는 데이터 출력 

# vice versa


print(set(object_train["rd_sd_nm"].unique())-set(object_val["rd_sd_nm"].unique()))

print(set(object_val["rd_sd_nm"].unique())-set(object_train["rd_sd_nm"].unique()))

print(set(object_train["rd_sd_nm"].unique())-set(object_test["rd_sd_nm"].unique()))

print(set(object_test["rd_sd_nm"].unique())-set(object_train["rd_sd_nm"].unique()))

crosstab(object_train,"rd_sd_nm")
crosstab_ratio(object_train,"rd_sd_nm")


road_low = ["광대로한면","맹지", "세로각지(가)", "세로각지(불)", "세로한면(가)", "세로한면(불)", "소로한면","지정되지않음", "소로각지"]
road_high = [ "광대세각", "광대소각","중로각지","중로한면"]


object_train['road_danger']=object_train.rd_sd_nm.apply(lambda x: "road_low" if x in road_low else x)
object_train['road_danger']=object_train.road_danger.apply(lambda x: "road_high" if x in road_high else x)

object_val['road_danger']=object_val.rd_sd_nm.apply(lambda x: "road_low" if x in road_low else x)
object_val['road_danger']=object_val.road_danger.apply(lambda x: "road_high" if x in road_high else x)

object_test['road_danger']=object_test.rd_sd_nm.apply(lambda x: "road_low" if x in road_low else x)
object_test['road_danger']=object_test.road_danger.apply(lambda x: "road_high" if x in road_high else x)

### trgt_cnt 소방점검대상물기준

object_train.loc[object_train["trgt_crtr"]=="자동화재탐지설치대상", "auto"] = 1
object_train.loc[object_train["trgt_crtr"]!="자동화재탐지설치대상", "auto"] = 0
object_train.loc[object_train["trgt_crtr"]=="옥내소화전설치대상", "oknae"] = 1
object_train.loc[object_train["trgt_crtr"]!="옥내소화전설치대상", "oknae"] = 0
object_train.loc[object_train["trgt_crtr"]=="스프링클러,물분무등설치대상", "sprinkler"] = 1
object_train.loc[object_train["trgt_crtr"]!="스프링클러,물분무등설치대상", "sprinkler"] = 0

object_val.loc[object_val["trgt_crtr"]=="자동화재탐지설치대상", "auto"] = 1
object_val.loc[object_val["trgt_crtr"]!="자동화재탐지설치대상", "auto"] = 0
object_val.loc[object_val["trgt_crtr"]=="옥내소화전설치대상", "oknae"] = 1
object_val.loc[object_val["trgt_crtr"]!="옥내소화전설치대상", "oknae"] = 0
object_val.loc[object_val["trgt_crtr"]=="스프링클러,물분무등설치대상", "sprinkler"] = 1
object_val.loc[object_val["trgt_crtr"]!="스프링클러,물분무등설치대상", "sprinkler"] = 0

object_test.loc[object_test["trgt_crtr"]=="자동화재탐지설치대상", "auto"] = 1
object_test.loc[object_test["trgt_crtr"]!="자동화재탐지설치대상", "auto"] = 0
object_test.loc[object_test["trgt_crtr"]=="옥내소화전설치대상", "oknae"] = 1
object_test.loc[object_test["trgt_crtr"]!="옥내소화전설치대상", "oknae"] = 0
object_test.loc[object_test["trgt_crtr"]=="스프링클러,물분무등설치대상", "sprinkler"] = 1
object_test.loc[object_test["trgt_crtr"]!="스프링클러,물분무등설치대상", "sprinkler"] = 0

### us_yn 사용 여부

us_yn = pd.get_dummies(object_train["us_yn"])
object_train = object_train.join(us_yn)

us_yn = pd.get_dummies(object_val["us_yn"])
object_val = object_val.join(us_yn)

us_yn = pd.get_dummies(object_test["us_yn"])
object_test = object_test.join(us_yn)


### dngrs_thng_yn 위험물대상여부

object_train.loc[object_train["dngrs_thng_yn"].isnull(),"dngrs_thng"]= 0
object_train.loc[object_train["dngrs_thng_yn"].notnull(),"dngrs_thng"]= 1

crosstab_ratio(object_train,"dngrs_thng_yn")

object_val.loc[object_val["dngrs_thng_yn"].isnull(),"dngrs_thng"]= 0
object_val.loc[object_val["dngrs_thng_yn"].notnull(),"dngrs_thng"]= 1

object_test.loc[object_test["dngrs_thng_yn"].isnull(),"dngrs_thng"]= 0
object_test.loc[object_test["dngrs_thng_yn"].notnull(),"dngrs_thng"]= 1


object_train.drop(["dt_of_fr","bldng_us","bldng_archtctr","dt_of_athrztn","bldng_us_clssfctn","jmk","rgnl_ar_nm","rgnl_ar_nm2","lnd_us_sttn_nm","rd_sd_nm","emd_nm","mlt_us_yn","trgt_crtr","fr_fghtng_fclt_spcl_css_5_yn","fr_fghtng_fclt_spcl_css_6_yn"\

,"us_yn","dngrs_thng_yn","slf_fr_brgd_yn","blk_dngrs_thng_mnfctr_yn","cltrl_hrtg_yn","bldng_archt_danger","Bldng_old_YN","fr_yn"],axis=1,inplace=True)



object_val.drop(["dt_of_fr","bldng_us","bldng_archtctr","dt_of_athrztn","bldng_us_clssfctn","jmk","rgnl_ar_nm","rgnl_ar_nm2","lnd_us_sttn_nm","rd_sd_nm","emd_nm","mlt_us_yn","trgt_crtr","fr_fghtng_fclt_spcl_css_5_yn","fr_fghtng_fclt_spcl_css_6_yn"\

,"us_yn","dngrs_thng_yn","slf_fr_brgd_yn","blk_dngrs_thng_mnfctr_yn","cltrl_hrtg_yn","bldng_archt_danger","Bldng_old_YN","fr_yn",'교정및군사시설','기타콘크리트구조',\
                '묘지관련시설','방송통신시설','통나무구조'],axis=1,inplace=True)

object_test.drop(["dt_of_fr","bldng_us","bldng_archtctr","dt_of_athrztn","bldng_us_clssfctn","jmk","rgnl_ar_nm","rgnl_ar_nm2","lnd_us_sttn_nm","rd_sd_nm","emd_nm","mlt_us_yn","trgt_crtr","fr_fghtng_fclt_spcl_css_5_yn","fr_fghtng_fclt_spcl_css_6_yn"\

,"us_yn","dngrs_thng_yn","slf_fr_brgd_yn","blk_dngrs_thng_mnfctr_yn","cltrl_hrtg_yn","bldng_archt_danger","Bldng_old_YN",'기타콘크리트구조',],axis=1,inplace=True)

print(set(object_test.columns)-set(object_train.columns))
print(set(object_test.columns)-set(object_val.columns))
print(set(object_train.columns)-set(object_test.columns))
print(set(object_train.columns)-set(object_val.columns))
print(set(object_val.columns)-set(object_train.columns))
print(set(object_val.columns)-set(object_test.columns))


# 수치형 데이터 따로 정리

numeric_list=list(set(train.columns.tolist()) - set(object_list))

numeric_list=sorted(numeric_list)

numeric_train=train[numeric_list[:]]


numeric_list=list(set(val.columns.tolist()) - set(object_list))

numeric_list=sorted(numeric_list)

numeric_val=val[numeric_list[:]]



numeric_list=list(set(test.columns.tolist()) - set(object_list))

numeric_list=sorted(numeric_list)

numeric_test=test[numeric_list[:]]

numeric_train

### bldng_cnt 건물채수

small_crit=11
middle_crit=74

numeric_train.loc[numeric_train["bldng_cnt"]<small_crit, "bldng_cnt_size"] = "few"
numeric_train.loc[(numeric_train["bldng_cnt"]>=small_crit)&(numeric_train["bldng_cnt"]<middle_crit), "bldng_cnt_size"] = "soso"
numeric_train.loc[numeric_train["bldng_cnt"]>=middle_crit, "bldng_cnt_size"] = "many"

small_crit=11
middle_crit=74

numeric_train.loc[train["bldng_cnt"]<small_crit, "bldng_cnt_encoded"] = "small"
numeric_train.loc[(train["bldng_cnt"]>=small_crit)&(numeric_train["bldng_cnt"]<middle_crit), "bldng_cnt_encoded"] = "middle"
numeric_train.loc[train["bldng_cnt"]>=middle_crit, "bldng_cnt_encoded"] = "big"

crosstab_ratio(numeric_train,"bldng_cnt_encoded")

one_hot_bldng_cnt = pd.get_dummies(numeric_train["bldng_cnt_size"])
numeric_train = numeric_train.join(one_hot_bldng_cnt)




numeric_val.loc[numeric_val["bldng_cnt"]<small_crit, "bldng_cnt_size"] = "few"
numeric_val.loc[(numeric_val["bldng_cnt"]>=small_crit)&(numeric_val["bldng_cnt"]<middle_crit), "bldng_cnt_size"] = "soso"
numeric_val.loc[numeric_val["bldng_cnt"]>=middle_crit, "bldng_cnt_size"] = "many"

one_hot_bldng_cnt = pd.get_dummies(numeric_val["bldng_cnt_size"])
numeric_val = numeric_val.join(one_hot_bldng_cnt)



numeric_test.loc[numeric_test["bldng_cnt"]<small_crit, "bldng_cnt_size"] = "few"
numeric_test.loc[(numeric_test["bldng_cnt"]>=small_crit)&(numeric_test["bldng_cnt"]<middle_crit), "bldng_cnt_size"] = "soso"
numeric_test.loc[numeric_test["bldng_cnt"]>=middle_crit, "bldng_cnt_size"] = "many"

one_hot_bldng_cnt = pd.get_dummies(numeric_test["bldng_cnt_size"])
numeric_test = numeric_test.join(one_hot_bldng_cnt)

### bldng_ar & ttl_ar & lnd_ar 건물건축면적, 건물연면적, 토지면적

# val 값이 전처리 하기 전이 조금 더 잘나옴

train["bldng_ar"].head(10)



bldng_ar_notnull = train[train["bldng_ar"]!=0]
sns.lmplot(data=bldng_ar_notnull, x="bldng_ar", y="ttl_ar", hue="fr_yn", fit_reg=False)

low_bldng_ar = train[(train["bldng_ar"]<4000)&(train["bldng_ar"]>0)&(train["ttl_ar"]<3000)&(train["ttl_ar"]>0)]
low_bldng_ar.shape

sns.lmplot(data=low_bldng_ar, x="bldng_ar", y="ttl_ar", hue="fr_yn", fit_reg=False)

low_bldng_ar = train[(train["bldng_ar"]<4000)&(train["bldng_ar"]>0)
                     &(train["ttl_ar"]<3000)&(train["ttl_ar"]>0)
                     &(train["lnd_ar"]<4000)&(train["lnd_ar"]>0)]

sns.lmplot(data=low_bldng_ar, x="bldng_ar", y="lnd_ar", hue="fr_yn", fit_reg=False, size=10)

sns.lmplot(data=low_bldng_ar, x="ttl_ar", y="lnd_ar", hue="fr_yn", fit_reg=False, size=10)

low_low_bldng_ar = train[(train["bldng_ar"]<1500)&(train["bldng_ar"]>0)
                     &(train["ttl_ar"]<2000)&(train["ttl_ar"]>0)
                     &(train["lnd_ar"]<2000)&(train["lnd_ar"]>0)]
low_low_bldng_ar.shape

sns.lmplot(data=low_low_bldng_ar, x="bldng_ar", y="ttl_ar", hue="fr_yn", fit_reg=False, size=10)

train[train["bldng_ar"]==0].shape

train["bldng_ar"].describe()

train["ttl_ar"].mean()

train["ttl_ar"].median()

train.loc[train["bldng_ar"]==0, "bldng_ar"] = train["bldng_ar"].median()
train.loc[train["ttl_ar"]==0, "ttl_ar"] = train["ttl_ar"].median()

val.loc[val["bldng_ar"]==0, "bldng_ar"] = val["bldng_ar"].median()
val.loc[val["ttl_ar"]==0, "ttl_ar"] = val["ttl_ar"].median()

test.loc[test["bldng_ar"]==0, "bldng_ar"] = test["bldng_ar"].median()
test.loc[test["ttl_ar"]==0, "ttl_ar"] = test["ttl_ar"].median()


#train["floors"] = train["ttl_ar"] / train["bldng_ar"]
#train["floors"].head()

train = train.drop("lnd_ar", 1)
val = val.drop("lnd_ar", 1)
test = test.drop("lnd_ar", 1)

## part 3

### ele_energy_us_YYYYMM 전기 에너지 사용량 (YYYY년 M월)

### slf_fr_brgd_yn 자체소방대여부

### blk_dngrs_thng_mnfctr_yn 대량위험물제조소등여부

### cltrl_hrtg_yn 문화재여부






train=numeric_train.merge(object_train,left_index=True,right_index=True)
train=train.drop(['many','dt_of_athrztn','fr_yn',"bldng_cnt_size","road_danger"],axis=1)

val=numeric_val.merge(object_val,left_index=True,right_index=True)
val=val.drop(['dt_of_athrztn','fr_yn',"bldng_cnt_size","road_danger"],axis=1)

test=numeric_test.merge(object_test,left_index=True,right_index=True)
test=test.drop(['many','dt_of_athrztn','fr_yn',"bldng_cnt_size","road_danger"],axis=1)


print(set(train.columns)-set(val.columns))
print(set(train.columns)-set(test.columns))
print(set(val.columns)-set(train.columns))
print(set(val.columns)-set(test.columns))
print(set(test.columns)-set(train.columns))
print(set(test.columns)-set(val.columns))


train=train.rename(columns={"공동주택":"flats","공장":"factory","관광휴게시설":"tourism_center","교육연구시설":"lab","근린생활시설":"leisure_center","노유자시설":"child","단독주택":"single_house"\
                           ,"동.식물 관련시설":"farm","문화및집회시설":"culture_center","분뇨.쓰레기처리시설":"cleaning","숙박시설":"hotel","업무시설":"office","운동시설":"gym","위락시설":"play_ground"\
                           ,"위험물저장및처리시설":"danger_facility","의료시설":"hospital","자동차관련시설":"car","제1종근린생활시설":"no_1_leisure","제2종근린생활시설":"no_2_leisure"\
                           ,"종교시설":"relision","창고시설":"warehouse","판매시설":"shop"})
val=val.rename(columns={"공동주택":"flats","공장":"factory","관광휴게시설":"tourism_center","교육연구시설":"lab","근린생활시설":"leisure_center","노유자시설":"child","단독주택":"single_house"\
                           ,"동.식물 관련시설":"farm","문화및집회시설":"culture_center","분뇨.쓰레기처리시설":"cleaning","숙박시설":"hotel","업무시설":"office","운동시설":"gym","위락시설":"play_ground"\
                           ,"위험물저장및처리시설":"danger_facility","의료시설":"hospital","자동차관련시설":"car","제1종근린생활시설":"no_1_leisure","제2종근린생활시설":"no_2_leisure"\
                           ,"종교시설":"relision","창고시설":"warehouse","판매시설":"shop"})

test=test.rename(columns={"공동주택":"flats","공장":"factory","관광휴게시설":"tourism_center","교육연구시설":"lab","근린생활시설":"leisure_center","노유자시설":"child","단독주택":"single_house"\
                           ,"동.식물 관련시설":"farm","문화및집회시설":"culture_center","분뇨.쓰레기처리시설":"cleaning","숙박시설":"hotel","업무시설":"office","운동시설":"gym","위락시설":"play_ground"\
                           ,"위험물저장및처리시설":"danger_facility","의료시설":"hospital","자동차관련시설":"car","제1종근린생활시설":"no_1_leisure","제2종근린생활시설":"no_2_leisure"\
                           ,"종교시설":"relision","창고시설":"warehouse","판매시설":"shop"})


train.isna().sum()



train_comp=train.copy()
val_comp=val.copy()
test_comp=test.copy()

# Train


X_train = train.drop('target_value', axis=1)
y_train = train['target_value']
X_val = val.drop('target_value', axis=1)
y_val = val['target_value']
test = test


df_all = pd.concat([X_train, X_val, test])

categorical_cols = df_all.select_dtypes(['object']).columns

for col in categorical_cols:
    df_all[col] = pd.Categorical(df_all[col]).codes

X_train = df_all[:len(train)]
X_val = df_all[len(train):-len(test)]
test = df_all[-len(test):]

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

imp = IterativeImputer(missing_values=np.nan, sample_posterior=False, 
                                 max_iter=10, tol=0.001, 
                                 n_nearest_features=4, initial_strategy='median')

imp.fit(X_train)
X_train = pd.DataFrame(data=imp.transform(X_train), 
                             columns=X_train.columns.tolist(),
                             dtype='int')

imp.fit(X_val)
X_val = pd.DataFrame(data=imp.transform(X_val), 
                             columns=X_val.columns.tolist(),
                             dtype='int')
imp.fit(test)
test = pd.DataFrame(data=imp.transform(test), 
                             columns=test.columns.tolist(),
                             dtype='int')


X_train = X_train.fillna(-1)
X_val = X_val.fillna(-1)
test = test.fillna(-1)

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X_train,y_train)
# print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

feat_importances.sort_values(ascending=False)

중요_변수_리스트=feat_importances[feat_importances>0.001].index.tolist()

# 중요_변수_리스트.append("Use_T_YN")
# X_train=X_train[중요_변수_리스트]

# X_val=X_val[중요_변수_리스트]



from  sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

f1_score(y_val, y_pred)

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)

for train_index, test_index in sss.split(X_train, y_train):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    original_ytrain, original_ytest = y_train.iloc[train_index], y_train.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels


# Turn into an array
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values


original_ytrain=pd.Series(original_ytrain)
original_ytest=pd.Series(original_ytest)


# X_train.columns = [''] * len(X_train.columns)
# y_train.columns = [''] * len(y_train.columns)
# X_val.columns = [''] * len(X_val.columns)
# y_val.columns = [''] * len(y_val.columns)




from bayes_opt import BayesianOptimization

# from sklearn.metrics import f1_score

# create dataset for lightgbm
lgb_train = lgb.Dataset(original_Xtrain, original_ytrain)
lgb_eval = lgb.Dataset(original_Xtest, original_ytest, reference=lgb_train)

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

evals_result = {}

params = {
        'task': 'train',
        'objective': 'binary',    
        'metric': 'binary_error', 
        'verbose': 1
}

clf = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                feval=lgb_f1_score,
                evals_result=evals_result,
                early_stopping_rounds=10)

# clf = lgb.train(params, train_data, valid_sets=[val_data, train_data], valid_names=['val', 'train'], feval=lgb_f1_score, evals_result=evals_result)

lgb.plot_metric(evals_result, metric='f1')

예측 = clf.predict(X_val, num_iteration=clf.best_iteration)

sns.distplot(예측)

예측 = clf.predict(X_val, num_iteration=clf.best_iteration)
예측[예측>0.49]=1
예측[예측<=0.49]=0

정답=y_val


from sklearn.metrics import confusion_matrix

cm = confusion_matrix(정답, 예측)
print(cm)

plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Versicolor or Not Versicolor Confusion Matrix - Test Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()

import sklearn.metrics

print(sklearn.metrics.recall_score(정답,예측))
print(sklearn.metrics.precision_score(정답,예측))
print(sklearn.metrics.accuracy_score(정답,예측))
print("####################################################")

print("이게중요!",sklearn.metrics.f1_score(정답,예측))

print("####################################################")

print(sklearn.metrics.precision_recall_curve(정답,예측))

# 베이지안 옵티미제이션

from bayes_opt import BayesianOptimization
import numpy as np

def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
#         params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.6, 1),
#                                             'max_depth': (5, 8.99),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (1e-5, 0.1),
                                            'min_child_weight': (0.5, 50)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO.res

opt_params = bayes_parameter_opt_lgb(X_train, y_train, init_round=7, opt_round=20, n_folds=5, random_seed=10, n_estimators=10000, learning_rate=0.05)

optimal_params = {v:float(t) for v,t in opt_params[16]['params'].items()}


optimal_params

optimal_params['num_leaves']=int(round(optimal_params['num_leaves']))
# optimal_params['max_depth']=int(round(optimal_params['max_depth']))

optimal_params.update({'task': 'train','objective': 'binary','metric': 'binary_error','verbose':1,'max_depth':-1})

total_data_X=X_train.append(X_val)

total_data_y=y_train.append(y_val)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


evals_result = {}

model = lgb.train(optimal_params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                feval=lgb_f1_score,
                evals_result=evals_result,
                early_stopping_rounds=5)



# clf = lgb.train(params, train_data, valid_sets=[val_data, train_data], valid_names=['val', 'train'], feval=lgb_f1_score, evals_result=evals_result)

# lgb.plot_metric(evals_result, metric='f1')

예측 = model.predict(X_val, num_iteration=model.best_iteration)

예측 = model.predict(X_val, num_iteration=model.best_iteration)
예측[예측>0.49]=1
예측[예측<=0.49]=0

정답=y_val

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(정답, 예측)
print(cm)

plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Versicolor or Not Versicolor Confusion Matrix - Test Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()

import sklearn.metrics

print(sklearn.metrics.recall_score(정답,예측))
print(sklearn.metrics.precision_score(정답,예측))
print(sklearn.metrics.accuracy_score(정답,예측))
print("####################################################")

print("이게중요!",sklearn.metrics.f1_score(정답,예측))

print("####################################################")

print(sklearn.metrics.precision_recall_curve(정답,예측))

X_test=test[중요_변수_리스트]

예측 = clf.predict(test, num_iteration=clf.best_iteration)
예측[예측>0.49]=1
예측[예측<=0.49]=0


예측=pd.DataFrame(예측,columns=["fr_yn"])

binary_y = {0: 'N', 1: 'Y'}

예측['fr_yn'] = 예측['fr_yn'].map(binary_y)

예측.index.drop()

예측['fr_yn'].to_csv("화재예측과제_Submission.csv")

예측.to_csv("화재예측과제_Submission.csv")

# train엔 있고 val엔 없는 데이터 출력 

# vice versa상", "sprinkler"] = 0

### us_yn 사용 여부

us_yn = pd.get_dummies(object_train["us_yn"])
object_train = object_train.join(us_yn)

us_yn = pd.get_dummies(object_val["us_yn"])
object_val = object_val.join(us_yn)

us_yn = pd.get_dummies(object_test["us_yn"])
object_test = object_test.join(us_yn)


### dngrs_thng_yn 위험물대상여부

object_train.loc[object_train["dngrs_thng_yn"].isnull(),"dngrs_thng"]= 0
object_train.loc[object_train["dngrs_thng_yn"].notnull(),"dngrs_thng"]= 1

crosstab_ratio(object_train,"dngrs_thng_yn")

object_val.loc[object_val["dngrs_thng_yn"].isnull(),"dngrs_thng"]= 0
object_val.loc[object_val["dngrs_thng_yn"].notnull(),"dngrs_thng"]= 1

object_test.loc[object_test["dngrs_thng_yn"].isnull(),"dngrs_thng"]= 0
object_test.loc[object_test["dngrs_thng_yn"].notnull(),"dngrs_thng"]= 1


object_train.drop(["dt_of_fr","bldng_us","bldng_archtctr","dt_of_athrztn","bldng_us_clssfctn","jmk","rgnl_ar_nm","rgnl_ar_nm2","lnd_us_sttn_nm","rd_sd_nm","emd_nm","mlt_us_yn","trgt_crtr","fr_fghtng_fclt_spcl_css_5_yn","fr_fghtng_fclt_spcl_css_6_yn"\

,"us_yn","dngrs_thng_yn","slf_fr_brgd_yn","blk_dngrs_thng_mnfctr_yn","cltrl_hrtg_yn","bldng_archt_danger","Bldng_old_YN","fr_yn"],axis=1,inplace=True)



object_val.drop(["dt_of_fr","bldng_us","bldng_archtctr","dt_of_athrztn","bldng_us_clssfctn","jmk","rgnl_ar_nm","rgnl_ar_nm2","lnd_us_sttn_nm","rd_sd_nm","emd_nm","mlt_us_yn","trgt_crtr","fr_fghtng_fclt_spcl_css_5_yn","fr_fghtng_fclt_spcl_css_6_yn"\

,"us_yn","dngrs_thng_yn","slf_fr_brgd_yn","blk_dngrs_thng_mnfctr_yn","cltrl_hrtg_yn","bldng_archt_danger","Bldng_old_YN","fr_yn",'교정및군사시설','기타콘크리트구조',\
                '묘지관련시설','방송통신시설','통나무구조'],axis=1,inplace=True)

object_test.drop(["dt_of_fr","bldng_us","bldng_archtctr","dt_of_athrztn","bldng_us_clssfctn","jmk","rgnl_ar_nm","rgnl_ar_nm2","lnd_us_sttn_nm","rd_sd_nm","emd_nm","mlt_us_yn","trgt_crtr","fr_fghtng_fclt_spcl_css_5_yn","fr_fghtng_fclt_spcl_css_6_yn"\

,"us_yn","dngrs_thng_yn","slf_fr_brgd_yn","blk_dngrs_thng_mnfctr_yn","cltrl_hrtg_yn","bldng_archt_danger","Bldng_old_YN",'기타콘크리트구조',],axis=1,inplace=True)

print(set(object_test.columns)-set(object_train.columns))
print(set(object_test.columns)-set(object_val.columns))
print(set(object_train.columns)-set(object_test.columns))
print(set(object_train.columns)-set(object_val.columns))
print(set(object_val.columns)-set(object_train.columns))
print(set(object_val.columns)-set(object_test.columns))


# 수치형 데이터 따로 정리

numeric_list=list(set(train.columns.tolist()) - set(object_list))

numeric_list=sorted(numeric_list)

numeric_train=train[numeric_list[:]]


numeric_list=list(set(val.columns.tolist()) - set(object_list))

numeric_list=sorted(numeric_list)

numeric_val=val[numeric_list[:]]



numeric_list=list(set(test.columns.tolist()) - set(object_list))

numeric_list=sorted(numeric_list)

numeric_test=test[numeric_list[:]]

numeric_train

### bldng_cnt 건물채수

small_crit=11
middle_crit=74

numeric_train.loc[numeric_train["bldng_cnt"]<small_crit, "bldng_cnt_size"] = "few"
numeric_train.loc[(numeric_train["bldng_cnt"]>=small_crit)&(numeric_train["bldng_cnt"]<middle_crit), "bldng_cnt_size"] = "soso"
numeric_train.loc[numeric_train["bldng_cnt"]>=middle_crit, "bldng_cnt_size"] = "many"

small_crit=11
middle_crit=74

numeric_train.loc[train["bldng_cnt"]<small_crit, "bldng_cnt_encoded"] = "small"
numeric_train.loc[(train["bldng_cnt"]>=small_crit)&(numeric_train["bldng_cnt"]<middle_crit), "bldng_cnt_encoded"] = "middle"
numeric_train.loc[train["bldng_cnt"]>=middle_crit, "bldng_cnt_encoded"] = "big"

crosstab_ratio(numeric_train,"bldng_cnt_encoded")

one_hot_bldng_cnt = pd.get_dummies(numeric_train["bldng_cnt_size"])
numeric_train = numeric_train.join(one_hot_bldng_cnt)




numeric_val.loc[numeric_val["bldng_cnt"]<small_crit, "bldng_cnt_size"] = "few"
numeric_val.loc[(numeric_val["bldng_cnt"]>=small_crit)&(numeric_val["bldng_cnt"]<middle_crit), "bldng_cnt_size"] = "soso"
numeric_val.loc[numeric_val["bldng_cnt"]>=middle_crit, "bldng_cnt_size"] = "many"

one_hot_bldng_cnt = pd.get_dummies(numeric_val["bldng_cnt_size"])
numeric_val = numeric_val.join(one_hot_bldng_cnt)



numeric_test.loc[numeric_test["bldng_cnt"]<small_crit, "bldng_cnt_size"] = "few"
numeric_test.loc[(numeric_test["bldng_cnt"]>=small_crit)&(numeric_test["bldng_cnt"]<middle_crit), "bldng_cnt_size"] = "soso"
numeric_test.loc[numeric_test["bldng_cnt"]>=middle_crit, "bldng_cnt_size"] = "many"

one_hot_bldng_cnt = pd.get_dummies(numeric_test["bldng_cnt_size"])
numeric_test = numeric_test.join(one_hot_bldng_cnt)

### bldng_ar & ttl_ar & lnd_ar 건물건축면적, 건물연면적, 토지면적

# val 값이 전처리 하기 전이 조금 더 잘나옴

train["bldng_ar"].head(10)



bldng_ar_notnull = train[train["bldng_ar"]!=0]
sns.lmplot(data=bldng_ar_notnull, x="bldng_ar", y="ttl_ar", hue="fr_yn", fit_reg=False)

low_bldng_ar = train[(train["bldng_ar"]<4000)&(train["bldng_ar"]>0)&(train["ttl_ar"]<3000)&(train["ttl_ar"]>0)]
low_bldng_ar.shape

sns.lmplot(data=low_bldng_ar, x="bldng_ar", y="ttl_ar", hue="fr_yn", fit_reg=False)

low_bldng_ar = train[(train["bldng_ar"]<4000)&(train["bldng_ar"]>0)
                     &(train["ttl_ar"]<3000)&(train["ttl_ar"]>0)
                     &(train["lnd_ar"]<4000)&(train["lnd_ar"]>0)]

sns.lmplot(data=low_bldng_ar, x="bldng_ar", y="lnd_ar", hue="fr_yn", fit_reg=False, size=10)

sns.lmplot(data=low_bldng_ar, x="ttl_ar", y="lnd_ar", hue="fr_yn", fit_reg=False, size=10)

low_low_bldng_ar = train[(train["bldng_ar"]<1500)&(train["bldng_ar"]>0)
                     &(train["ttl_ar"]<2000)&(train["ttl_ar"]>0)
                     &(train["lnd_ar"]<2000)&(train["lnd_ar"]>0)]
low_low_bldng_ar.shape

sns.lmplot(data=low_low_bldng_ar, x="bldng_ar", y="ttl_ar", hue="fr_yn", fit_reg=False, size=10)

train[train["bldng_ar"]==0].shape

train["bldng_ar"].describe()

train["ttl_ar"].mean()

train["ttl_ar"].median()

train.loc[train["bldng_ar"]==0, "bldng_ar"] = train["bldng_ar"].median()
train.loc[train["ttl_ar"]==0, "ttl_ar"] = train["ttl_ar"].median()

val.loc[val["bldng_ar"]==0, "bldng_ar"] = val["bldng_ar"].median()
val.loc[val["ttl_ar"]==0, "ttl_ar"] = val["ttl_ar"].median()

test.loc[test["bldng_ar"]==0, "bldng_ar"] = test["bldng_ar"].median()
test.loc[test["ttl_ar"]==0, "ttl_ar"] = test["ttl_ar"].median()


#train["floors"] = train["ttl_ar"] / train["bldng_ar"]
#train["floors"].head()

train = train.drop("lnd_ar", 1)
val = val.drop("lnd_ar", 1)
test = test.drop("lnd_ar", 1)

## part 3

### ele_energy_us_YYYYMM 전기 에너지 사용량 (YYYY년 M월)

### slf_fr_brgd_yn 자체소방대여부

### blk_dngrs_thng_mnfctr_yn 대량위험물제조소등여부

### cltrl_hrtg_yn 문화재여부






train=numeric_train.merge(object_train,left_index=True,right_index=True)
train=train.drop(['many','dt_of_athrztn','fr_yn',"bldng_cnt_size","road_danger"],axis=1)

val=numeric_val.merge(object_val,left_index=True,right_index=True)
val=val.drop(['dt_of_athrztn','fr_yn',"bldng_cnt_size","road_danger"],axis=1)

test=numeric_test.merge(object_test,left_index=True,right_index=True)
test=test.drop(['many','dt_of_athrztn','fr_yn',"bldng_cnt_size","road_danger"],axis=1)


print(set(train.columns)-set(val.columns))
print(set(train.columns)-set(test.columns))
print(set(val.columns)-set(train.columns))
print(set(val.columns)-set(test.columns))
print(set(test.columns)-set(train.columns))
print(set(test.columns)-set(val.columns))


train=train.rename(columns={"공동주택":"flats","공장":"factory","관광휴게시설":"tourism_center","교육연구시설":"lab","근린생활시설":"leisure_center","노유자시설":"child","단독주택":"single_house"\
                           ,"동.식물 관련시설":"farm","문화및집회시설":"culture_center","분뇨.쓰레기처리시설":"cleaning","숙박시설":"hotel","업무시설":"office","운동시설":"gym","위락시설":"play_ground"\
                           ,"위험물저장및처리시설":"danger_facility","의료시설":"hospital","자동차관련시설":"car","제1종근린생활시설":"no_1_leisure","제2종근린생활시설":"no_2_leisure"\
                           ,"종교시설":"relision","창고시설":"warehouse","판매시설":"shop"})
val=val.rename(columns={"공동주택":"flats","공장":"factory","관광휴게시설":"tourism_center","교육연구시설":"lab","근린생활시설":"leisure_center","노유자시설":"child","단독주택":"single_house"\
                           ,"동.식물 관련시설":"farm","문화및집회시설":"culture_center","분뇨.쓰레기처리시설":"cleaning","숙박시설":"hotel","업무시설":"office","운동시설":"gym","위락시설":"play_ground"\
                           ,"위험물저장및처리시설":"danger_facility","의료시설":"hospital","자동차관련시설":"car","제1종근린생활시설":"no_1_leisure","제2종근린생활시설":"no_2_leisure"\
                           ,"종교시설":"relision","창고시설":"warehouse","판매시설":"shop"})

test=test.rename(columns={"공동주택":"flats","공장":"factory","관광휴게시설":"tourism_center","교육연구시설":"lab","근린생활시설":"leisure_center","노유자시설":"child","단독주택":"single_house"\
                           ,"동.식물 관련시설":"farm","문화및집회시설":"culture_center","분뇨.쓰레기처리시설":"cleaning","숙박시설":"hotel","업무시설":"office","운동시설":"gym","위락시설":"play_ground"\
                           ,"위험물저장및처리시설":"danger_facility","의료시설":"hospital","자동차관련시설":"car","제1종근린생활시설":"no_1_leisure","제2종근린생활시설":"no_2_leisure"\
                           ,"종교시설":"relision","창고시설":"warehouse","판매시설":"shop"})


train.isna().sum()



train_comp=train.copy()
val_comp=val.copy()
test_comp=test.copy()

# Train


X_train = train.drop('target_value', axis=1)
y_train = train['target_value']
X_val = val.drop('target_value', axis=1)
y_val = val['target_value']
test = test


df_all = pd.concat([X_train, X_val, test])

categorical_cols = df_all.select_dtypes(['object']).columns

for col in categorical_cols:
    df_all[col] = pd.Categorical(df_all[col]).codes

X_train = df_all[:len(train)]
X_val = df_all[len(train):-len(test)]
test = df_all[-len(test):]

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

imp = IterativeImputer(missing_values=np.nan, sample_posterior=False, 
                                 max_iter=10, tol=0.001, 
                                 n_nearest_features=4, initial_strategy='median')

imp.fit(X_train)
X_train = pd.DataFrame(data=imp.transform(X_train), 
                             columns=X_train.columns.tolist(),
                             dtype='int')

imp.fit(X_val)
X_val = pd.DataFrame(data=imp.transform(X_val), 
                             columns=X_val.columns.tolist(),
                             dtype='int')
imp.fit(test)
test = pd.DataFrame(data=imp.transform(test), 
                             columns=test.columns.tolist(),
                             dtype='int')


X_train = X_train.fillna(-1)
X_val = X_val.fillna(-1)
test = test.fillna(-1)

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X_train,y_train)
# print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

feat_importances.sort_values(ascending=False)

중요_변수_리스트=feat_importances[feat_importances>0.001].index.tolist()

# 중요_변수_리스트.append("Use_T_YN")
# X_train=X_train[중요_변수_리스트]

# X_val=X_val[중요_변수_리스트]



from  sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

f1_score(y_val, y_pred)

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)

for train_index, test_index in sss.split(X_train, y_train):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    original_ytrain, original_ytest = y_train.iloc[train_index], y_train.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels


# Turn into an array
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values


original_ytrain=pd.Series(original_ytrain)
original_ytest=pd.Series(original_ytest)


# X_train.columns = [''] * len(X_train.columns)
# y_train.columns = [''] * len(y_train.columns)
# X_val.columns = [''] * len(X_val.columns)
# y_val.columns = [''] * len(y_val.columns)




from bayes_opt import BayesianOptimization

# from sklearn.metrics import f1_score

# create dataset for lightgbm
lgb_train = lgb.Dataset(original_Xtrain, original_ytrain)
lgb_eval = lgb.Dataset(original_Xtest, original_ytest, reference=lgb_train)

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

evals_result = {}

params = {
        'task': 'train',
        'objective': 'binary',    
        'metric': 'binary_error', 
        'verbose': 1
}

clf = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                feval=lgb_f1_score,
                evals_result=evals_result,
                early_stopping_rounds=10)

# clf = lgb.train(params, train_data, valid_sets=[val_data, train_data], valid_names=['val', 'train'], feval=lgb_f1_score, evals_result=evals_result)

lgb.plot_metric(evals_result, metric='f1')

예측 = clf.predict(X_val, num_iteration=clf.best_iteration)

sns.distplot(예측)

예측 = clf.predict(X_val, num_iteration=clf.best_iteration)
예측[예측>0.49]=1
예측[예측<=0.49]=0

정답=y_val


from sklearn.metrics import confusion_matrix

cm = confusion_matrix(정답, 예측)
print(cm)

plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Versicolor or Not Versicolor Confusion Matrix - Test Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()

import sklearn.metrics

print(sklearn.metrics.recall_score(정답,예측))
print(sklearn.metrics.precision_score(정답,예측))
print(sklearn.metrics.accuracy_score(정답,예측))
print("####################################################")

print("이게중요!",sklearn.metrics.f1_score(정답,예측))

print("####################################################")

print(sklearn.metrics.precision_recall_curve(정답,예측))

# 베이지안 옵티미제이션

from bayes_opt import BayesianOptimization
import numpy as np

def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
#         params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.6, 1),
#                                             'max_depth': (5, 8.99),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (1e-5, 0.1),
                                            'min_child_weight': (0.5, 50)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO.res

opt_params = bayes_parameter_opt_lgb(X_train, y_train, init_round=7, opt_round=20, n_folds=5, random_seed=10, n_estimators=10000, learning_rate=0.05)

optimal_params = {v:float(t) for v,t in opt_params[16]['params'].items()}


optimal_params

optimal_params['num_leaves']=int(round(optimal_params['num_leaves']))
# optimal_params['max_depth']=int(round(optimal_params['max_depth']))

optimal_params.update({'task': 'train','objective': 'binary','metric': 'binary_error','verbose':1,'max_depth':-1})

total_data_X=X_train.append(X_val)

total_data_y=y_train.append(y_val)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


evals_result = {}

model = lgb.train(optimal_params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                feval=lgb_f1_score,
                evals_result=evals_result,
                early_stopping_rounds=5)



# clf = lgb.train(params, train_data, valid_sets=[val_data, train_data], valid_names=['val', 'train'], feval=lgb_f1_score, evals_result=evals_result)

# lgb.plot_metric(evals_result, metric='f1')

예측 = model.predict(X_val, num_iteration=model.best_iteration)

예측 = model.predict(X_val, num_iteration=model.best_iteration)
예측[예측>0.49]=1
예측[예측<=0.49]=0

정답=y_val

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(정답, 예측)
print(cm)

plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Versicolor or Not Versicolor Confusion Matrix - Test Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()

import sklearn.metrics

print(sklearn.metrics.recall_score(정답,예측))
print(sklearn.metrics.precision_score(정답,예측))
print(sklearn.metrics.accuracy_score(정답,예측))
print("####################################################")

print("이게중요!",sklearn.metrics.f1_score(정답,예측))

print("####################################################")

print(sklearn.metrics.precision_recall_curve(정답,예측))

X_test=test[중요_변수_리스트]

예측 = clf.predict(test, num_iteration=clf.best_iteration)
예측[예측>0.49]=1
예측[예측<=0.49]=0


예측=pd.DataFrame(예측,columns=["fr_yn"])

binary_y = {0: 'N', 1: 'Y'}

예측['fr_yn'] = 예측['fr_yn'].map(binary_y)

예측.index.drop()

예측['fr_yn'].to_csv("화재예측과제_Submission.csv")

예측.to_csv("화재예측과제_Submission.csv")